In [1]:
import pandas as pd
import numpy as np
import scipy

In [2]:
ind_data=pd.read_csv('industry_data/ind_data.csv',header=0)

In [3]:
def org_data(ind_list, index_list, data_frame):
    new_frame=pd.DataFrame()
    for i in range(len(ind_list)):
        for j in range(len(index_list)):
            if j==0:
                new_frame['%s_%s'%(ind_list[i],index_list[j])] = data_frame['%s'%ind_list[i]]
            else:
                new_frame['%s_%s'%(ind_list[i],index_list[j])] = data_frame['%s.%s'%(ind_list[i],j)]
    return new_frame

In [4]:
def modify_data(data_frame, industry_list, index_list):
    for item in index_list:
        for ind in industry_list:
            data_frame['%s_%s'%(ind,item)]=(data_frame['%s_%s'%(ind,item)]/data_frame['%s_C_P'%ind])**(-1)
    return data_frame

In [5]:
def diff(data_frame, diff_index):
    diff=pd.DataFrame(new_frame)
    for item in diff_index:
        diff[str(item)]=np.diff(data_frame[str(item)])/data_frame[str(item)][:len(data_frame[str(item)])-1]
    return diff

In [6]:
def normalize(data):
    data_new=data/(data[np.argmax(data)]-data[np.argmin(data)])
    return data_new

In [7]:
industry_list=['Energy','Materials','Industrials','Consumer Discretionary','Consumer Staples','Healthcare','Financials','IT','Telecom Services','Utilities']

index_list=['C_P','Vol','PE','PB','PS','PCF']

new_frame=org_data(industry_list, index_list, ind_data)
new_frame=modify_data(new_frame, industry_list, ['PE','PB','PS','PCF'])


HS300=pd.read_excel('HS300.xlsx')


new_frame['HS300']=HS300['Close']

droplist=[]
for i in industry_list:
    droplist.append('%s_C_P'%i)

#new_frame_undrop=diff(new_frame, new_frame.keys()).dropna()
#new_frame_undrop.to_csv('industry_data/undrop_diff_ind.csv',index=False, header=True)

new_frame=new_frame.drop(droplist,axis=1)

new_frame.to_csv('industry_data/org_ind_data.csv', header=True)

In [8]:
import neurolab as nl

In [9]:
diff_data=diff(new_frame, new_frame.keys())
diff_data_ad=pd.DataFrame(diff_data.dropna())
diff_data_ad.to_csv('industry_data/diff_data.csv')

In [10]:
diff_data.dropna().shape

(689, 51)

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

/Users/liyuefan/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [12]:
diff_data_ad['HS300']=map(lambda x: 1 if x>=0 else -1, diff_data_ad['HS300'])

In [13]:
diff_data_ad.shape

(689, 51)

Then normalize data

In [14]:
norm_diff_data=pd.DataFrame()
for key in diff_data_ad.keys():
    if key=='HS300':
        norm_diff_data[str(key)]=diff_data_ad[str(key)]
    else:
        norm_diff_data[str(key)]=normalize(diff_data_ad[str(key)])

In [15]:
norm_diff_data.to_csv('industry_data/data_for_nn.csv',header=True,index=False)

In [16]:
input_nn=norm_diff_data.drop('HS300',axis=1)

In [17]:
input_nn

,Energy_Vol,Energy_PE,Energy_PB,Energy_PS,Energy_PCF,Materials_Vol,Materials_PE,Materials_PB,Materials_PS,Materials_PCF,...,Telecom Services_Vol,Telecom Services_PE,Telecom Services_PB,Telecom Services_PS,Telecom Services_PCF,Utilities_Vol,Utilities_PE,Utilities_PB,Utilities_PS,Utilities_PCF
0,0.319191,-0.001158,0.010535,-0.057060,0.000097,-0.009068,-0.001070,0.108408,-0.010525,0.000992,...,-0.030368,-0.001565,-0.125099,0.126323,-0.000317,0.021887,0.059837,0.037704,0.076076,1.301153e-03
1,-0.128985,-0.008224,0.021049,0.007824,-0.000198,-0.194069,-0.004340,-0.052830,0.029600,-0.001240,...,-0.070138,0.002408,0.094742,-0.033755,0.000090,-0.241776,-0.052157,0.038399,-0.108808,-1.115016e-03
2,-0.154923,0.000625,-0.042012,0.046029,-0.000388,-0.111909,-0.027571,-0.081109,-0.021222,-0.000871,...,-0.060223,0.001102,0.021484,-0.050894,0.000689,-0.087289,0.004902,0.000000,0.000000,1.738168e-04
3,0.304654,0.002498,0.031638,-0.049341,0.000291,0.240303,0.000802,-0.039835,-0.045307,-0.000185,...,0.128897,0.002360,-0.053309,0.148742,0.000191,0.262473,0.335169,-0.090225,0.027119,-4.321280e-04
4,-0.094288,-0.001029,-0.001267,0.038071,-0.000191,-0.158836,-0.002687,0.027986,-0.032453,-0.000735,...,-0.049191,-0.004969,-0.033021,-0.120934,-0.001359,-0.245072,-0.013784,-0.053243,0.010558,-3.483174e-04
5,0.000191,-0.006260,-0.040793,-0.053175,-0.000981,-0.151626,0.010010,0.028248,0.051716,0.000467,...,-0.092459,0.003351,0.076696,0.017600,0.000521,-0.133554,0.011100,0.065237,-0.022340,2.654151e-04
6,0.008487,0.004178,0.052784,0.019619,0.000489,0.252114,-0.010370,-0.083381,-0.021817,0.000141,...,0.235866,0.002826,0.053877,0.070211,0.000937,0.351460,0.065288,-0.034490,-0.049605,-3.531232e-04
7,-0.054994,0.002857,-0.066201,-0.009508,-0.000590,-0.082946,0.017321,0.029045,0.016530,0.000478,...,-0.083778,0.033216,0.045544,-0.104204,0.000298,0.024490,0.017699,-0.003782,0.066909,2.635725e-04
8,-0.068460,-0.004426,0.033515,0.035791,-0.000292,-0.067484,0.002770,0.013843,-0.041446,-0.001341,...,0.013107,-0.000039,-0.175390,0.017647,-0.000619,-0.029798,-0.027893,-0.041078,-0.010408,-1.753083e-04
9,0.184159,0.002094,0.002776,-0.014384,0.000396,0.191240,-0.024965,-0.098220,0.006204,-0.000583,...,0.115048,0.002312,0.108432,0.140797,0.000733,0.142367,0.540256,0.052508,-0.010132,0.000000e+00


In [18]:
input_nn.shape

(689, 50)

In [19]:
train_sample=np.array(input_nn[:500]).reshape(500,50)
test_sample=np.array(input_nn[501:688]).reshape(187,50)

In [20]:
target_nn=np.array(norm_diff_data['HS300'])

In [21]:
train_target=target_nn[1:501].reshape(500,1)
test_target=target_nn[502:689].reshape(187,1)

In [109]:

# Create network with 50 inputs, 5 neurons in input layer and 1 in output layer
net = nl.net.newelm([[-1, 1], [-1, 1],[-1, 1], [-1, 1],
                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],
                    [-1, 1], [-1, 1],[-1, 1], [-1, 1],[-1, 1], 
                    [-1, 1],[-1, 1], [-1, 1],[-1, 1], [-1, 1],[-1, 1], [-1, 1],[-1, 1], 
                    [-1, 1],[-1, 1], [-1, 1],[-1, 1], [-1, 1],[-1, 1], [-1, 1],[-1, 1], [-1, 1]], 
                   [10,1])

net.layers[0].initf = nl.init.InitRand([-0.1, 0.1], 'wb')
net.layers[1].initf= nl.init.InitRand([-0.1, 0.1], 'wb')
net.init()

net.train(train_sample,train_target, epochs=1000, goal=0.01, show=15)
# Train process
#err = net.train(train_sample, train_target, show=15)



Epoch: 15; Error: 0.988123469206;
Epoch: 30; Error: 1.06787057759;
Epoch: 45; Error: 0.963278399875;
Epoch: 60; Error: 0.947062859844;
Epoch: 75; Error: 0.940086874703;
Epoch: 90; Error: 0.942920330897;
Epoch: 105; Error: 0.937955958258;
Epoch: 120; Error: 0.94726578892;
Epoch: 135; Error: 1.01801382938;
Epoch: 150; Error: 0.946274170914;
Epoch: 165; Error: 0.933380116528;
Epoch: 180; Error: 0.965828160991;
Epoch: 195; Error: 0.976994857109;
Epoch: 210; Error: 0.940135642499;
Epoch: 225; Error: 0.951074910678;
Epoch: 240; Error: 0.936696584518;
Epoch: 255; Error: 0.970607099051;
Epoch: 270; Error: 0.960391511212;
Epoch: 285; Error: 0.909278842847;
Epoch: 300; Error: 0.911480166411;
Epoch: 315; Error: 0.895163588052;
Epoch: 330; Error: 0.884599241617;
Epoch: 345; Error: 0.926130528658;
Epoch: 360; Error: 0.922916903996;
Epoch: 375; Error: 0.879762605988;
Epoch: 390; Error: 0.857461231044;
Epoch: 405; Error: 0.855745477614;
Epoch: 420; Error: 0.85782554036;
Epoch: 435; Error: 0.832746793

[1.0023931580272534,
 0.99882007566176001,
 1.0016795521614623,
 0.99971477134157594,
 0.99818048636607526,
 0.99959087182089756,
 0.99651480071761844,
 0.99704602761632011,
 0.99565997666902017,
 0.99396426036345686,
 0.99378903841708133,
 0.99125749061957946,
 0.99118968750332759,
 0.98871226102316034,
 0.98812346920622829,
 0.98581705388815399,
 0.98308990944438412,
 0.98087744275825017,
 0.97655922879819157,
 0.9715068463315174,
 0.96694817192652771,
 0.96298627486101762,
 0.96120450225748244,
 1.0077086109515299,
 1.311462233022225,
 0.95414209335868083,
 1.1352376597716369,
 1.0021512887162707,
 1.0232713920719956,
 1.0678705775874384,
 1.0520931365173525,
 1.0892572817091299,
 1.0278300912063298,
 1.1942911084684789,
 1.2455769254777582,
 1.0862905754736571,
 1.0014161716848036,
 1.0205248379126501,
 1.032260878420965,
 1.0165993069067705,
 0.99711660486023446,
 0.98023613225832007,
 0.96922147181188667,
 0.96614776770086375,
 0.96327839987451036,
 0.95781546770203729,
 0.957630

In [110]:
predict=net.sim(test_sample)

In [111]:
predict

array([[-0.78674939],
       [ 0.3886979 ],
       [ 0.17514911],
       [ 0.34645739],
       [-0.03922855],
       [-0.30503451],
       [ 0.17538805],
       [ 0.43703843],
       [-0.73760461],
       [ 0.29536   ],
       [ 0.46094163],
       [ 0.56309133],
       [ 0.15111268],
       [-0.99977792],
       [-0.60783117],
       [-0.02771766],
       [ 0.35593136],
       [ 0.30869015],
       [ 0.98340979],
       [ 0.796216  ],
       [ 0.99937661],
       [ 0.8968775 ],
       [ 0.12867259],
       [ 0.42876846],
       [ 0.1370176 ],
       [ 0.18751239],
       [-0.48628525],
       [-0.98929426],
       [ 0.70825015],
       [ 0.21831633],
       [ 0.12687967],
       [ 0.33202423],
       [-0.47982052],
       [ 0.5024438 ],
       [-0.20943076],
       [ 0.36533579],
       [ 0.04298128],
       [-0.38719811],
       [ 0.19926236],
       [ 0.10228664],
       [-0.97330472],
       [ 0.99202576],
       [ 0.20645867],
       [-0.5332237 ],
       [-0.83095503],
       [ 0

In [112]:
predict=map(lambda x: 1 if x>=0 else -1, predict)

In [113]:
predict

[-1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1]

In [114]:
count=0
for i in range(len(predict)):
    if predict[i]==test_target[i]:
        count+=1
    else:
        continue

In [115]:
count

101

In [116]:
float(count)/len(predict)

0.5401069518716578